# JuliaPatents + Lens.org demo

(c) 2023 Alex Seitz <ei271@uni-heidelberg.de>

JuliaPatents is a Julia package ecosystem currently in development for patent data analysis.
It currently provides support for [Lens.org](https://lens.org) as a data source,
and will eventually also support [PatStat](https://www.epo.org/searching-for-patents/business/patstat.html).

## Components

The ecosystem consists of the following packages:
- *PatentsBase* for general interface functions
- *PatentsLandscapes* for analysis functions and custom taxonomies
- *PatentsLens* for use of the Lens.org data source

## Scope of this demo

This demo is focused on the SQLite data model.
See the [official PatentsBase documentation](https://juliapatents.github.io/PatentsBase.jl/) for information on the object model and general functionality.

## 0. Preparation

### Subscribing to the registry

JuliaPatents packages are currently not in the general registry.
Therefore, the JuliaPatents registry must be added once on every device.

In [17]:
using Pkg
pkg"registry add https://github.com/JuliaPatents/Registry"

     Cloning registry from "https://github.com/JuliaPatents/Registry"


Registry `JuliaPatents` already exists in `~/.julia/registries/JuliaPatents`.


### Acquiring the packages

In [18]:
Pkg.add("PatentsBase")
Pkg.add("PatentsLandscapes")
Pkg.add("PatentsLens")

   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


### Loading the packages

In [2]:
using PatentsBase, PatentsLens, PatentsLandscapes

## 1. Loading data into the database

### Acquiring data

Data should be downloaded from [Lens.org](https://lens.org) in the JSON Lines (.jsonl) format.
For this demo, we'll be using a dataset of technical textiles patents.
We will download the 50,000 most relevant patent documents related to the search term "technical textile" from Lens.
Note: To download more than 1,000 documents at a time, a free Lens.org account is required.

### Creating a database

PatentsLens uses an SQLite backend, so there is no database server and databases are simply files.

In [3]:
demo_db = LensDB("demo.db")

LensDB(SQLite.DB("demo.db"))

### Loading the data

This step is quite computationally expensive as the data needs to transformed.
It may take several minutes for larger datasets such as this one.
The `skip_on_error` flag allows the loading process to skip malformatted lines in the data rather than fail when it encounters an error.

In [21]:
PatentsLens.load_jsonl!(demo_db, "technical-textile-50000.jsonl", skip_on_error = true)

Processing chunk #1 (app #1 - #5000)


Processing chunk #2 (app #5001 - #10000)


Processing chunk #3 (app #10001 - #15000)


Processing chunk #4 (app #15001 - #20000)


Processing chunk #5 (app #20001 - #25000)


...

Processing chunk #7 (app #30001 - #35000)


Processing chunk #8 (app #35001 - #40000)


Processing chunk #9 (app #40001 - #45000)


Processing chunk #10 (app #45001 - #50000)


Processing chunk #11 (app #50001 - #55000)


### Checking if it worked

(the syntax for this command will be explained further below)

In [5]:
prepdata(demo_db, Frequency(), ApplicationLevel())

Row,applications
,Int64
1,49999


## 2. Simple frequency analyses

The main function for database-backed analyses is called [`prepdata`](https://juliapatents.github.io/PatentsLandscapes.jl/dev/#PatentsLandscapes.prepdata).
As its first argument, it takes a data source, in this case the database we created.
The second argument is the analysis type, in this case `Frequency()`.
The third argument is the analysis level, which can be `ApplicationLevel()` or `FamilyLevel()`.
Arguments after this can be filters and groupings.
The code above is the most simple example, it simply counts the applications in the database.
We can also do this with families:

In [6]:
prepdata(demo_db, Frequency(), FamilyLevel())

Row,families
,Int64
1,41591


Simple families are usually a better unit of analysis than applications, since each simple families roughly corresponds to an invention.
Hence, all examples here will use families.

### Groupings

Frequency analysis starts becoming useful when we can differentiate across groups.
The prepdata command allows us to specify a grouping as the fourth argument:

In [7]:
res = prepdata(demo_db, Frequency(), FamilyLevel(), Jurisdictions())

Row,jurisdiction,families
,String,Int64
1,AU,18
2,BE,1
3,BG,7
4,CA,28
5,CH,8
6,CN,32077
7,CS,2
8,CZ,5
9,DE,90


The above example gives us the number of patent families in the dataset for each jurisdiction.
We can sort the output to get a better view of the important ones:

In [8]:
sort(res, :families, rev = true)

Row,jurisdiction,families
,String,Int64
1,CN,32077
2,US,6599
3,WO,3403
4,EP,2487
5,RU,229
6,DE,90
7,GE,38
8,CA,28
9,ES,28


For most groupings (not just jurisdictions), we can also specify which groups we're interested in:

In [9]:
res = prepdata(demo_db, Frequency(), FamilyLevel(), Jurisdictions(["CN", "US", "WO", "RU", "JP"]))

Row,jurisdiction,families
,String,Int64
1,CN,32077
2,JP,7
3,RU,229
4,US,6599
5,WO,3403


Other groupings currently available are Applicants, Time Trends, and custom Taxonomies (see chapter 5).
More grouping options will be added as the package matures.

In [10]:
prepdata(demo_db, Frequency(), FamilyLevel(), Applicants()) |> 
res -> sort(res, :families, rev = true) |> 
res -> first(res, 10)

Row,applicant_id,country,name,families
,Int64,String,String,Int64
1,744,??,UNIV JIANGNAN,350
2,852,??,UNIV WUHAN TEXTILE,285
3,2764,??,LUTHAI TEXTILE CO LTD,156
4,22,US,MILLIKEN & CO,155
5,7,US,NIKE INC,134
6,4297,DE,HENKEL AG & CO KGAA,118
7,1824,??,LUFENG KNITTING & DYEING CO LTD,105
8,4535,??,JIHUA 3542 TEXTILE CO LTD,105
9,299,DE,BASF SE,79


In [11]:
using Dates
prepdata(demo_db, Frequency(), FamilyLevel(), TimeTrend(start = Date("2005"), stop = Date("2020"), resolution = Months()))

Row,month,families
,String,Int64
1,2005-01,29
2,2005-02,21
3,2005-03,21
4,2005-04,16
5,2005-05,19
6,2005-06,24
7,2005-07,24
8,2005-08,25
9,2005-09,20


### Multidimensional frequency analyses

Frequencies can be broken down across multiple groupings. Currently, up to three groupings are possible.

Here's an analysis with two groupings:

In [12]:
prepdata(
    demo_db, 
    Frequency(), 
    FamilyLevel(), 
    Jurisdictions(["CN", "US", "WO"]), 
    TimeTrend(start = Date("2010"), stop = Date("2016"), resolution = Years()))

Row,jurisdiction,year,families
,String,String,Int64
1,CN,2010,332
2,CN,2011,341
3,CN,2012,626
4,CN,2013,789
5,CN,2014,801
6,CN,2015,892
7,US,2010,192
8,US,2011,191
9,US,2012,215


And here's one with three:

In [13]:
prepdata(
    demo_db, 
    Frequency(), 
    FamilyLevel(), 
    Jurisdictions(["CN", "US", "WO"]), 
    TimeTrend(start = Date("2010"), stop = Date("2016"), resolution = Years()),
    Applicants([744, 852, 2764, 22, 5]))

Row,jurisdiction,year,applicant_id,country,name,families
,String,String,Int64,String,String,Int64
1,CN,2010,744,??,UNIV JIANGNAN,11
2,CN,2011,744,??,UNIV JIANGNAN,35
3,CN,2011,852,??,UNIV WUHAN TEXTILE,9
4,CN,2011,2764,??,LUTHAI TEXTILE CO LTD,1
5,CN,2012,744,??,UNIV JIANGNAN,39
6,CN,2012,852,??,UNIV WUHAN TEXTILE,9
7,CN,2013,744,??,UNIV JIANGNAN,36
8,CN,2013,852,??,UNIV WUHAN TEXTILE,21
9,CN,2014,744,??,UNIV JIANGNAN,9


## 3. Filters

Filters allow subsetting of databases based on composable conditions.
They are useful for efficient scanning and selection of data and are a main building block of other concepts like taxonomies.
The database is optimized with lookup tables to apply these filters very quickly.

### Classification Filters

We can create a filter based on [IPC](https://ipcpub.wipo.int/) or [CPC](https://worldwide.espacenet.com/patent/cpc-browser#) classifications, at any level of the classification.

In [14]:
filter1 = ClassificationFilter(IPC(), Section(), IPCSymbol.(["A"]))
filter2 = ClassificationFilter(CPC(), Class(), IPCSymbol.(["D01", "D21"]))

ClassificationFilter(CPC(), Class(), IPCSymbol[IPCSymbol("D01"), IPCSymbol("D21")])

### Applying filters in an analysis

`prepdata()` allows specifying a filter after the analysis level to specify the subset of the database to be analyzed.

In [15]:
prepdata(demo_db, Frequency(), FamilyLevel(), filter1) # Count number of patent families in IPC section A "Human Necessities" in the database

Row,families
,Int64
1,4752


In a grouped analysis, the filter is passed before the groupings.

In [16]:
prepdata(demo_db, Frequency(), FamilyLevel(), filter1, TimeTrend(start = Date("2010")))

Row,year,families
,String,Int64
1,2010,129
2,2011,124
3,2012,133
4,2013,155
5,2014,159
6,2015,192
7,2016,189
8,2017,252
9,2018,336


### Content Filters

The main strength of the database model is indexation.
A search index allows for very fast search in the content fields of documents:
title, abstract, claims and full text.

The following filter allows us to select patent families mentioning antibacterial properties:

In [19]:
filter3 = ContentFilter("anti bacterial OR anti septic", [TitleSearch(), AbstractSearch(), ClaimsSearch(), FulltextSearch()])

UnionFilter(UnionFilter(UnionFilter(ContentFilter("anti bacterial OR anti septic", TitleSearch(), String[]), ContentFilter("anti bacterial OR anti septic", AbstractSearch(), String[])), ContentFilter("anti bacterial OR anti septic", ClaimsSearch(), String[])), ContentFilter("anti bacterial OR anti septic", FulltextSearch(), String[]))

In [20]:
prepdata(demo_db, Frequency(), FamilyLevel(), filter3, Applicants()) |> res -> sort(res, :families, rev = true)

Row,applicant_id,country,name,families
,Int64,String,String,Int64
1,4297,DE,HENKEL AG & CO KGAA,27
2,299,DE,BASF SE,15
3,7449,US,PROCTER & GAMBLE,15
4,433,BE,JANSSEN PHARMACEUTICA NV,10
5,4543,??,MAURER KARL-HEINZ,10
6,10114,DK,NOVOZYMES AS,10
7,116,DE,HENKEL KGAA,9
8,4544,??,KOTTWITZ BEATRIX,8
9,25138,BE,BYLEMANS DANY LEOPOLD JOZEFIEN,8


As we can see, the leading innovators in this area are different ones than for the overall dataset.

### Composing filters

Filters can be composed using the union operator `|` or the intersection operator `&`.
Let's say we're interested in methods for producing antibacterial fibers.
We could use a content filter with an `AND` operator:

In [21]:
filter4 = ContentFilter("anti bacterial AND fiber", [TitleSearch(), AbstractSearch(), ClaimsSearch(), FulltextSearch()])
prepdata(demo_db, Frequency(), FamilyLevel(), filter4)

Row,families
,Int64
1,271


But this would include all applications mentioning "fiber", not just those dealing with how to produce it.
Alternatively, we can filter for CPC class D01, referring to "NATURAL OR MAN-MADE THREADS OR FIBRES; SPINNING".

In [23]:
filter5 = filter3 & ClassificationFilter(CPC(), Class(), [CPCSymbol("D01")]) # filter3 is our anti bacterial filter from earlier
prepdata(demo_db, Frequency(), FamilyLevel(), filter5)

Row,families
,Int64
1,48


As we can see, this filter is much more specific.

In the same way, we can use the union operator to make our filter more general.
For instance, this filter matches all data either classified in CPC class D01, or mentioning the word "fiber".

In [24]:
filter6 = 
    ClassificationFilter(CPC(), Class(), [CPCSymbol("D01")]) |
    ContentFilter("fiber", [TitleSearch(), AbstractSearch(), ClaimsSearch(), FulltextSearch()])
prepdata(demo_db, Frequency(), FamilyLevel(), filter6)

Row,families
,Int64
1,9390


## 4. Taxonomies

Taxonomies combine the filter and grouping interfaces to allow for very flexible analysis specifications.
In essence, a taxonomy is a set of filter results that is indexed for fast reaccess and can be used as a grouping.
Taxonomies are created using the `define_taxon` function from PatentsLandscapes.

Say we want to create a taxonomy for special properties of technical textiles.
We simply write a filter for each taxon, and then call `define_taxon` on it:

In [25]:
define_taxon!(demo_db, "special_properties", 
"antibacterial",
    filter3)

define_taxon!(demo_db, "special_properties", 
"antistatic",
    ContentFilter("anti static", 
        [TitleSearch(), AbstractSearch(), ClaimsSearch(), FulltextSearch()]))

define_taxon!(demo_db, "special_properties", 
"chemical_resistant",
    ContentFilter("chemical resistant OR acid resistant OR anti corrosive OR corrosion resistant",
        [TitleSearch(), AbstractSearch(), ClaimsSearch(), FulltextSearch()]))

define_taxon!(demo_db, "special_properties", 
"flame_retardant",
    ContentFilter("flame retardant OR flame resistant OR fire resistant OR fire retardant OR fire proof OR non inflammable",
        [TitleSearch(), AbstractSearch(), ClaimsSearch(), FulltextSearch()]))

define_taxon!(demo_db, "special_properties", 
"insect_repellant",
    ContentFilter("insect OR mosquito", 
        [TitleSearch(), AbstractSearch(), ClaimsSearch(), FulltextSearch()]))

define_taxon!(demo_db, "special_properties", 
"anti_odor",
    ContentFilter("odor absorbent OR odor resistant",
        [TitleSearch(), AbstractSearch(), ClaimsSearch(), FulltextSearch()]))

define_taxon!(demo_db, "special_properties", 
"uv_resistant",
    ContentFilter("uv resistant OR uv reflective OR ultraviolet resistant OR ultraviolet reflective",
        [TitleSearch(), AbstractSearch(), ClaimsSearch(), FulltextSearch()]))

SQLite.Query{false}(SQLite.Stmt(SQLite.DB("demo.db"), Base.RefValue{Ptr{SQLite.C.sqlite3_stmt}}(Ptr{SQLite.C.sqlite3_stmt} @0x000000000ce71fd8), Dict{Int64, Any}(2 => "uv_resistant", 1 => "special_properties")), Base.RefValue{Int32}(101), Symbol[], Type[], Dict{Symbol, Int64}(), Base.RefValue{Int64}(0))

We can then use the taxonomy as a grouping for our analysis:

In [26]:
prepdata(demo_db, Frequency(), FamilyLevel(), Taxonomy("special_properties"))

Row,special_properties,families
,String,Int64
1,anti_odor,277
2,antibacterial,479
3,antistatic,841
4,chemical_resistant,1877
5,flame_retardant,1746
6,insect_repellant,316
7,uv_resistant,814


The original filter result can also be conveniently re-accessed using a `TaxonomicFilter`:

In [27]:
filter7 = TaxonomicFilter("special_properties", ["antistatic"])
prepdata(demo_db, Frequency(), FamilyLevel(), filter7, TimeTrend())

Row,year,families
,String,Int64
1,1950,1
2,1976,1
3,1977,7
4,1978,3
5,1979,4
6,1980,2
7,1981,2
8,1982,5
9,1983,1


### Orthogonal and nested taxonomies

There can be multiple taxonomies in a database.
For instance, we may want to combine the special properties taxonomy with one for different types of biopolymers.
Taxonomies can be nested by simply creating different sub-taxonomies.
This code (taken from the PatentsLens test suite) generates a two-level nested biopolymer taxonomy:

In [43]:
polymers = [
    "Drop-Ins" => [
        "Bio-PET" => "bio pet OR bio polyethylene terephthalate"
        "Bio-PE"  => "bio pe OR bio polyethylene"
        "Bio-PUR" => "bio pur OR bio polyurethane"
        "Bio-PA"  => "bio pa OR bio polyamide"
    ],
    "Cellulose-based" => [
        "cellulose acetate" => "cellulose acetate"
        "viscose / rayon"   => "viscose OR rayon"
        "methyl cellulose"  => "methyl cellulose"
        "ethyl cellulose"   => "ethyl cellulose"
    ],
    "PLA" => ["PLA" => "pla OR polylactic acid"],
    "PGA" => ["PGA" => "pga OR polylglycolide OR glycolic acid OR polyglycolic acid"],
    "PHA/PHB/PHV" => ["PHA/PHB/PHV" => "pha OR phb OR phv OR polyhydroxybutyrate OR polyhydroxyvalerate OR polyhydroxyalkanoate OR polyhydroxy butyrate OR polyhydroxy valerate OR polyhydroxy alkanoate"],
    "PBS" => ["PBS" => "phb OR polybutylene succinate"],
    "PBAT" => ["PBAT" => "pbat OR polybutylene adipate terephthalate"],
    "PEF" => ["PEF" => "pef OR polyethylene furanoate"]
]

for level1 in polymers
    level1_name = level1.first
    for level2 in level1.second
        level2_name = level2.first
        terms = level2.second
        define_taxon!(
            demo_db,
            "polymers2",
            level2_name,
            ContentFilter(terms, [TitleSearch(), AbstractSearch(), ClaimsSearch(), FulltextSearch()]),
            expand = false)
    end
    level2_names = (l2 -> l2.first).(level1.second)
    define_taxon!(
        demo_db,
        "polymers1",
        level1_name,
        TaxonomicFilter("polymers2", level2_names),
        expand = false)
end

Let's see a frequency count for the general types of biopolymers (level 1):

In [44]:
prepdata(demo_db, Frequency(), FamilyLevel(), Taxonomy("polymers1"))

Row,polymers1,families
,String,Int64
1,Cellulose-based,3165
2,Drop-Ins,317
3,PBAT,57
4,PBS,69
5,PEF,20
6,PGA,258
7,PHA/PHB/PHV,115
8,PLA,530


Assume we're interested in the subtypes of cellulose-based biopolymers and their special properties.
We can combine the two taxonomies in a frequency analysis, like any other groupings:

In [48]:
prepdata(
    demo_db,
    Frequency(),
    FamilyLevel(), 
    TaxonomicFilter("polymers1", ["Cellulose-based"]), 
    Taxonomy("polymers2", ["cellulose acetate", "viscose / rayon", "methyl cellulose", "ethyl cellulose"]), 
    Taxonomy("special_properties"))

Row,polymers2,special_properties,families
,String,String,Int64
1,cellulose acetate,anti_odor,79
2,cellulose acetate,antibacterial,138
3,cellulose acetate,antistatic,121
4,cellulose acetate,chemical_resistant,385
5,cellulose acetate,flame_retardant,235
6,cellulose acetate,insect_repellant,81
7,cellulose acetate,uv_resistant,176
8,ethyl cellulose,anti_odor,65
9,ethyl cellulose,antibacterial,105


## 5. Returning to the object model

Sometimes during exploration, a part of the data may be so interesting that we want to have a closer look.
To do this, we can pull the result of any filter from the database and get back the original object model, which we can then inspect.
Note that because this requires us to reverse the Object-Relational Mapping (ORM), it may take a few minutes depending on how much data is extracted. 

In [50]:
filter8 = TaxonomicFilter("polymers1", ["Cellulose-based"]) & TaxonomicFilter("special_properties", ["flame_retardant"])
fams = families(demo_db, filter8)

581-element Vector{LensFamily}:
 LensFamily(LensApplication[000-310-338-078-328 | 2020-12-22 | US10870768B2, 000-310-338-078-328 | 2020-12-22 | US10870768B2])
 LensFamily(LensApplication[000-724-328-555-506 | 2016-11-08 | US9485990B2, 000-724-328-555-506 | 2016-11-08 | US9485990B2, 101-466-697-696-913 | 2015-03-05 | US20150064226A1, 106-537-696-923-245 | 2011-10-20 | US20110256198A1, 187-509-617-345-257 | 2014-12-09 | US8906398B2])
 LensFamily(LensApplication[000-919-196-122-556 | 2013-02-13 | EP2334491B1, 000-919-196-122-556 | 2013-02-13 | EP2334491B1, 076-244-845-658-671 | 2010-04-15 | WO2010042465A1, 111-858-535-811-902 | 2011-07-14 | US20110171867A1])
 LensFamily(LensApplication[001-045-494-297-313 | 2006-09-28 | WO2006101933A2, 001-045-494-297-313 | 2006-09-28 | WO2006101933A2, 099-666-304-484-845 | 2006-09-21 | US20060211319A1])
 LensFamily(LensApplication[001-077-883-141-538 | 2009-12-10 | US20090305025A1, 001-077-883-141-538 | 2009-12-10 | US20090305025A1, 027-822-370-504-963 |

In [56]:
app = applications(fams[1])[1] # get the first application from the first family extracted
title(app) |> println
abstract(app) |> println

(en) Composition for making coated yarn


(en) An aqueous composition is used to clad yarn cores to provide unique coated yarns. This aqueous composition contains: (i) porous particles present in an amount of at least 2 weight % and up to and including 10 weight %, each porous particle comprising a continuous polymeric phase and discrete pores dispersed within the continuous polymeric phase, the porous particles having a mode particle size of 2-50 μm; (ii) a film-forming binder material having a Tg of less than or equal to 25° C., that is present in an amount of 25-60 weight %; (iii) an inorganic filler material having a value of less than 5 on the MOHS scale of mineral hardness, in an amount of at least 2-15 weight %; and (iv) an aqueous medium in an amount of at least 35 weight % in which the film-forming binder material is soluble or dispersible.
